In [1]:
import cv2
import os
import re
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix

In [2]:
def atoi(text) : 
    return int(text) if text.isdigit() else text

In [3]:
def natural_keys(text) :
    return [atoi(c) for c in re.split('(\d+)', text)]

In [4]:
def getFilenames(path):
    root_path=""
    filenames = []
    for root, dirnames, filenames in os.walk(path):
        filenames.sort(key = natural_keys)
        rootpath = root
    return rootpath,filenames

In [5]:
def readImages(rootpath,filenames):
    images = []
    for filename in filenames :
        filepath = os.path.join(root,filename)
        image = ndimage.imread(filepath, mode = "L")
        images.append(image)
    return images

In [6]:
brain_filenames = []
face_filenames = []
motorbike_filenames = []
brain_images = []
face_images = []
motorbike_images = []

In [7]:
root,brain_filenames = getFilenames("/home/ashritha/14CO121_Assignment3/brain")
brain_images = readImages(root,brain_filenames)
root,face_filenames = getFilenames("/home/ashritha/14CO121_Assignment3/Faces_easy")
face_images = readImages(root,face_filenames)
root, motorbike_filenames = getFilenames("/home/ashritha/14CO121_Assignment3/Motorbikes")
motorbike_images = readImages(root,motorbike_filenames)

In [8]:
print(len(brain_images))
print(len(face_images))
print(len(motorbike_images))

98
435
798


In [9]:
def chooseRandom(images):
    train_indices = np.random.choice(len(images),int(0.7*len(images)),replace = False)
    train_images = []
    test_images = []
    for i in train_indices:
        train_images.append(images[i])
    test_indices = [x for x in range(len(images)) if x not in train_indices]
    for i in test_indices:
        test_images.append(images[i])
    return train_images,test_images

In [10]:
brain_train = []
brain_test = []
face_train = []
face_test = []
motorbike_train = []
motorbike_test = []

In [11]:
brain_train,brain_test = chooseRandom(brain_images)
face_train,face_test = chooseRandom(face_images)
motorbike_train,motorbike_test = chooseRandom(motorbike_images)

In [12]:
print(len(brain_train))
print(len(face_train))
print(len(motorbike_train))

68
304
558


In [13]:
print(len(brain_test))
print(len(face_test))
print(len(motorbike_test))

30
131
240


In [14]:
train_images = {}
train_images['brain'] = brain_train
train_images['face'] = face_train
train_images['motorbike'] = motorbike_train

In [15]:
test_images = {}
test_images['brain'] = brain_test
test_images['face'] = face_test
test_images['motorbike'] = motorbike_test

In [16]:
num_images = len(train_images['brain'])+len(train_images['face'])+len(train_images['motorbike'])

In [17]:
num_images

930

In [18]:
def extract_features_sift(image):
    sift_object = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift_object.detectAndCompute(image, None)
    return descriptors

In [19]:
def kmeans(descriptor,number_of_clusters):
    feature_size = descriptor[0]
    vStack = np.array(feature_size)
    for remaining in descriptor_list[1:]:
        vStack = np.vstack((vStack, remaining))
    formatted_descriptor = vStack.copy()
    k_means_object = KMeans(n_clusters=number_of_clusters,n_jobs=-1)
    k_means_ret_obj = k_means_object.fit_predict(formatted_descriptor)
    return [k_means_object,k_means_ret_obj]

In [20]:
def bag_of_words(descriptor_list,k_means_ret_obj,number_of_clusters,number_of_images):
    final_histogram = np.array([np.zeros(number_of_clusters) for i in range(number_of_images)])
    count = 0
    for i in range(number_of_images):
        length = len(descriptor_list[i])
        for j in range(length):
            idx = k_means_ret_obj[count+j]
            final_histogram[i][idx] += 1
        count += length
    print ("Vocabulary Histogram Generated")
    return final_histogram

In [21]:
def scale_histogram(final_histogram):
    scale = StandardScaler().fit(final_histogram)
    final_histogram = scale.transform(final_histogram)
    return [scale,final_histogram]

In [22]:
def train(final_histogram,train_labels):
    classifier = GaussianNB()
    classifier.fit(final_histogram,train_labels)
    print("Training completed")
    return classifier

In [23]:
def predict_image(classfier,scale,k_means_object,test_image,number_of_clusters):
    desc = extract_features_sift(test_image)
    vocab = np.array( [[ 0 for i in range(number_of_clusters)]])
    k_means_test_ret_obj = k_means_object.predict(desc)
    for i in k_means_test_ret_obj:
            vocab[0][i] += 1
    print(vocab)
    vocab = scale.transform(vocab)
    label = classifier.predict(vocab)
    return label

In [24]:
train_labels = np.array([])
label_count = 0
label_encoding = {}
descriptor_list = []
NUM_CLUSTERS = 200
print(len(train_images))

for label,image_list in train_images.items():
    label_encoding[str(label_count)] = label
    for img in image_list:
        train_labels = np.append(train_labels,label_count)
        desc = extract_features_sift(img)
        descriptor_list.append(desc)
    label_count = label_count + 1

print(len(descriptor_list))
k_means_object,k_means_ret_obj = kmeans(descriptor_list,NUM_CLUSTERS)
number_of_images = len(descriptor_list)
final_histogram = bag_of_words(descriptor_list,k_means_ret_obj,NUM_CLUSTERS,number_of_images)
scale,final_histogram = scale_histogram(final_histogram)
classifier= train(final_histogram,train_labels)


3
930
Vocabulary Histogram Generated
Training completed


In [202]:
k_means_ret_obj = np.load('kmeans_return_120.npy')
final_histogram = np.load('final_hist_120.npy')
k_means_object = KMeans(120,n_jobs=-1)
NUM_CLUSTERS = 120

In [26]:
y_true = []
y_predicted = []

for label,image_list in train_images.items():
    print("Processing ",label)
    for img in image_list:
        predicted_label = predict_image(classifier,scale,k_means_object,img,NUM_CLUSTERS)
        print(predicted_label)
        y_true.append(label)
        y_predicted.append(label_encoding[str(int(predicted_label[0]))])
    print(len(y_true))
    print(len(y_predicted))
    score = accuracy_score(y_true,y_predicted)
    print("Accuracy ",score)
    conf_matrix = confusion_matrix(y_true,y_predicted)
    print("Confusion matrix ",conf_matrix)

Processing  motorbike
[[1 0 0 0 0 2 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 2 0 1 1 1 1 0 0 1 2 1 1 0 0
  0 1 0 0 1 2 0 1 0 0 0 1 0 0 0 1 2 1 1 0 0 3 1 0 1 1 0 0 2 0 2 0 3 0 1 0
  2 0 2 0 1 0 0 0 0 0 0 2 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 2 1 1 0 0 0 2 0
  0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 2 0 0 0 0 0 1 1 0 1 0 0 0 0 2 1 0 0 0 0 0
  0 0 0 2 1 0 3 1 1 1 0 1 0 0 0 0 3 2 1 1 0 1 3 0 1 0 0 4 0 0 0 1 1 0 0 1
  2 0 0 0 1 0 0 2 1 0 0 2 0 1 0 0 0 0 0 1]]
[0.]
[[0 2 0 0 2 1 0 1 0 0 0 1 0 0 1 2 0 1 0 1 0 4 2 2 2 2 0 2 1 1 1 0 2 1 0 1
  1 0 0 1 4 2 0 3 1 1 1 2 0 1 0 0 2 0 4 0 2 3 1 0 2 0 0 0 1 2 3 1 2 0 0 2
  1 2 0 0 2 1 0 1 0 0 3 1 0 2 2 0 2 0 0 0 1 1 0 0 1 0 0 0 0 0 3 1 0 2 0 1
  1 2 0 2 1 0 2 0 0 1 0 0 0 0 2 0 0 0 1 0 3 0 0 3 1 0 0 0 0 0 0 0 1 0 1 1
  2 0 1 5 0 1 0 2 0 3 0 2 2 0 0 0 4 1 0 1 1 1 0 0 0 3 2 6 0 0 2 1 0 1 1 1
  1 1 0 2 0 1 0 0 2 2 1 0 0 1 1 2 0 0 0 1]]
[0.]


/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


[[0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 2 1 1 3 0 1 1 0 0 0 0 0 0
  0 0 2 1 1 1 2 0 1 0 1 0 2 2 2 2 0 0 1 0 0 0 0 1 4 3 1 0 0 1 1 1 1 0 0 0
  3 0 0 0 3 0 0 1 1 0 0 1 1 1 0 0 2 0 1 1 1 0 1 1 2 2 0 1 2 1 0 0 0 0 0 2
  0 4 0 0 1 2 2 0 1 0 1 0 3 0 0 2 0 0 0 1 0 0 1 0 0 0 0 0 1 0 2 0 2 0 1 1
  0 0 1 3 1 0 0 1 0 1 1 4 0 0 0 2 0 1 2 1 1 0 0 0 1 0 0 2 1 0 2 1 0 0 1 1
  0 1 0 0 1 1 0 0 2 0 3 1 2 0 0 1 0 2 0 1]]
[0.]
[[ 1  2  2  2  6  1  1  1  3  2  6  0  0  0  1  3  7  2  1  1  3  1  1  4
   3  1  2  1  7  1  1  3  1  1  1  4  3  0  2  4  0  4  1  3  1  0  1  3
   5  3  2  2  0  1  1  2  4  2  1  0  3  0  4  1  1  2  2  1  1  2  0  3
   3  3  1  1  1  3  5  3  1  3  2  1  4  3  0  1  2  1  5  6  2  2  3  1
   0  3  4  1  1  4  0  1  1  2  0  0  0  1  1  1  3  2  2  1  2  2  0  1
   6  0  2  1  2  2  4  1  1  2  0  1  2  1  2  2  0  2  4 18  2  0  3  0
   2  3  1  1  3  0  2  0  3  2  4  1  1  2  2  1  5  2  0  3  2  3  2  1
   2  2  1  1  3  7  1  3  3  1  0  6  0  3  1  1  2  1  1  2  

[[2 5 1 1 1 2 4 4 0 1 3 5 0 4 3 0 6 2 1 2 6 2 2 1 3 0 4 4 3 0 0 6 3 1 0 3
  4 2 4 6 3 2 1 2 5 2 3 0 3 1 3 1 1 2 0 2 1 0 1 0 0 0 0 3 0 4 1 1 1 2 1 0
  0 1 2 2 2 1 0 6 0 1 1 3 1 0 3 1 1 2 1 1 1 1 0 3 1 1 1 0 0 1 1 3 2 0 1 2
  1 4 2 2 3 4 2 1 1 1 0 2 0 5 2 0 0 0 0 1 1 2 0 5 1 1 1 2 2 1 0 5 3 0 1 0
  2 1 0 4 3 1 2 0 1 4 3 2 2 0 0 1 3 2 0 1 2 0 5 1 1 3 0 1 0 1 3 3 5 0 0 0
  1 1 1 0 0 0 3 3 0 2 1 0 1 1 0 2 1 2 1 0]]
[0.]
[[1 2 0 1 0 0 1 1 3 1 0 1 2 0 0 2 3 0 2 1 2 0 1 1 0 0 0 0 3 0 2 6 2 0 1 7
  2 0 2 3 1 2 0 3 2 0 0 0 0 1 0 1 0 0 1 1 4 0 2 0 3 0 3 4 0 0 2 0 0 2 3 2
  2 0 0 0 3 3 1 2 2 1 0 0 1 1 0 2 3 2 1 0 2 1 1 2 3 4 0 2 1 0 0 1 3 2 2 0
  1 0 1 1 0 0 1 0 1 2 0 1 0 3 4 0 4 1 1 1 0 1 0 1 2 0 3 8 2 5 0 1 2 0 0 0
  2 1 0 0 1 1 0 1 3 2 1 0 1 2 0 1 0 0 2 1 0 3 2 0 4 3 2 3 4 3 0 0 4 0 3 2
  3 0 0 1 0 2 1 1 1 0 0 0 1 0 0 1 0 1 1 0]]
[0.]
[[8 0 1 0 0 0 0 0 3 1 0 0 0 1 0 0 7 0 1 1 0 0 0 4 0 4 0 2 0 2 0 7 1 4 2 1
  0 2 1 0 0 0 2 0 0 0 1 1 0 3 2 0 2 4 2 3 0 1 1 2 0 0 1 0 3 0 0 1 2 0 0 0
  5 2 0 1 1 2 

[[2 0 0 1 2 0 2 0 0 1 0 0 2 0 1 0 1 0 0 2 0 0 1 2 1 0 1 1 3 3 0 1 1 0 0 2
  0 1 0 1 0 0 0 1 2 1 0 1 1 0 0 5 0 0 2 1 0 1 1 0 0 0 0 0 0 1 1 1 3 0 1 1
  0 0 0 0 4 4 0 0 0 0 1 0 0 1 2 0 0 0 0 1 0 0 0 1 0 0 0 1 1 3 1 0 0 0 0 1
  0 3 0 1 2 1 0 0 0 5 4 0 0 0 4 1 2 0 1 3 1 2 0 1 1 0 0 0 0 0 2 1 0 0 0 2
  1 0 4 1 0 2 0 0 0 4 0 0 2 2 0 0 3 0 0 0 0 0 2 0 0 0 2 2 0 0 2 2 0 1 1 1
  1 2 1 0 1 0 1 0 1 0 1 0 2 0 0 0 1 2 0 0]]
[0.]
[[0 1 3 1 1 0 0 2 2 2 0 1 1 3 0 0 4 2 0 3 2 1 1 1 0 2 2 1 1 1 1 5 0 0 1 5
  0 2 1 1 0 0 1 2 2 0 2 3 0 2 0 3 1 2 0 0 2 4 1 0 0 0 2 4 2 1 5 2 0 0 0 2
  1 2 0 1 0 1 2 2 2 0 1 0 1 0 1 0 0 2 4 3 1 1 3 1 1 0 1 3 1 2 1 0 1 1 2 0
  0 0 1 0 3 0 0 0 5 1 3 0 3 0 0 3 7 1 1 2 0 3 2 0 0 1 2 5 1 2 0 0 0 5 3 3
  0 2 0 0 0 2 4 1 1 1 1 0 0 0 1 1 0 1 1 2 0 1 0 0 2 0 1 0 2 0 0 0 3 0 0 1
  1 2 0 0 2 4 0 5 1 2 1 0 2 1 2 0 0 0 0 0]]
[0.]
[[1 0 0 2 0 0 0 0 2 2 1 2 0 3 1 2 1 3 2 3 2 1 0 1 2 1 2 1 1 2 0 2 0 0 1 5
  0 0 0 0 0 1 2 2 0 0 1 1 1 1 2 3 0 4 3 4 1 3 2 2 0 1 1 0 0 1 3 0 1 1 2 4
  0 1 0 6 1 1 

...

[[ 6  1  3 15  0  2  1  3 27 10  1  2  3 11  0  2  0  1  5  2  1  1  2  1
   0  3  1  2  0  2 13  0 10  5  1  2  1 21  2  3  0  0  2  3  2  2  7  1
   0  6  1  1  5 11  4  2  2 10  2  0  5  2  5  1 27  1  5  7  1  0  2  3
   9  6  6 11  0  1  0  0  2  2  5  0  3  3  0  1  7  6  2  3  4  9 11  5
   2  0  0  6  3  1  1  3  6  3  2  0  3  0  4  0  9  3  0  0  3  0  2  0
   7  2  1  3 15  3  5  5  1  4  0  5  1  4  7  3  3  0  1  0  3  1 12  6
   2  1  0  1  0  5  5  5  4  2  6  1  1  0  0  0  0  3  0  4  1 19  0  7
   5  1  1  0  4  0  0  0  2  2  2  0  1  2  3  1  1  7  0  7  1  1  0  8
   3  2  7  0  1  2  1  6]]
[2.]
[[ 1 16  5  5  6  2  0  9  0  1  0  1 10  1  5  7  0 18  3  1  5  0  1  1
   5  1  6  2  0  1  2  0  5  0  4  0 18  0  0  9  4 12  0  5  2  8  0  1
   1  6  2  1  9  0  1  1  9  9  0  0  6  8  0  1  2  8  2  0 12  7  4  3
   1  2  9  2  1  8  0  0 10  2  2  5  0  1  2  9  0 13  1  0  8  5  5 13
   6  2 11  1  1  6  4  3  2  3  0  1  8  0  9 19  4  7  1  9  1  0  0  4
   1 

[[ 0  9  1  0  0  4  0 13  1  1  0  2  5  1  6  1  0 11  3  2  4  2  3  1
   2  2 14  1  6  0  3  0  9  1  1  0 10  0  2  5  2  6  0  0  3  1  2  1
   2  3  2  1  1  2  3  2  3 10  1  1  8 12  0  2  0 12  5  1  1  5  6  5
   2  0  2  2  3  1  0  4 15  2  2  2  1  0  5 19  0  8  5  0  4  2  2  5
   0  1  2  0  0  5  2  0  6  2  0  5 14  0  5  8  0  9  0 11  1  3  0  1
   1  3  0  2  1  5  1  0  5  0  2  4  0  0  2  3  4  1  8  0  5  4  0  0
   0  0  7  0  1  0  3  4  2  0  3 10 10  1  1  2  1  9  5  2  2  2  0  3
   3  9  2  0  1 10 13 18  3  2  4  1  2  0  1  1  1  2  9  4  4 13  0  4
   2  3  0  4  2  4  3  1]]
[2.]
[[ 1 11  4  0  6  2  4 10  1  0  1  0  1  0  7  4  1 12  1  3  5  5  0  1
  14  1 11  0  1  7  0  2  3  2  0  0  7  0  0  1  2 22  1  1  1  3  0  1
   2  4  2  3  6  1 10  1  4  3  0  3  8 16  0  1  0  9  1  0  3  4  4  1
   0  3  1  0  1  0  0  3  4  2  1 17  0  2  6 19  1  3  4  0  6  3  1  4
   3  0  7  0  1  3 16  0  2  2  4  4 14  3  4 25  0  1  0  4  1  1  3  0
   0 

[[ 0 10  1  0 19  2  5  6  0  0  0  0  2  0  5  4  1  6  3  0  1  3  0  0
  11  0 14  0  0  2  0  1  4  0  3  0  1  0  1  0  5 14  2  0  2  3  0  0
   0  1  1  5  6  0  0  0  1  3  0  0  1 13  3  0  0  2  0  1 16  4  4  0
   0  5  0  0  3 13  0  0  3  1  0 11  1  1  2 10  0  6  0  0  6  0  2  5
   2  0  4  1  1  1  6  2  1  2  0  3  8  0  4 21  0  1  1  3  2  1  1  1
   1  1  0  2  0  0  1  8 11  1  0  0  1  0  1  0  0  0  1  4  3  0  0  0
   3  0  2 13  0 27  1  8  3  1  4  3  4  0  0  6  2 11  1  1  1  0  1  1
   2 17  0  3  1  2  2  9  2  1  0  0  2  7  0  4  0  0 12  2  1  1  2  0
   3  4  0  5  0  1  0  2]]
[2.]
[[ 1 14  2  3 16  2  1 16  1  1  1  0  3  2  9  6  0 18  3  3  0  1  7  1
  10  5 14  1  0  9  0  0  2  2  4  0 19  1  2  1  7 15  0  3  3  2  3  1
   1  5  2  1  2  2  0  2  4  3  1  2  7 12  1  1  0  5  2  1 14  1  2  1
   3  3  5  1  1  3  0  4  2  0  1  6  2  3  3 32  0  5  2  1  3  2  2 12
   1  0  2  1  5 10 14  2  2  7  1  1  9  3  7 20  1  2  1  6  0  8  2  6
   2 

[[ 2  4  2  1  4  3  6  1  3  5  1  3  0  1  2  2  0  9  1  4  0  4  2  2
   3  4  3  1  0  6  3  0  4  1  5  0  4  0  3  3  5  4  2  1  1  2  4  1
   1  3  1  3  2  1  1  4  4  3  2  3  2  1  3  5  1  7  2  4  3  0  1  3
   4  2  2  1  0  0  0  3  3  4  3  9  3  3  6  4  1  2  0  3  1  2  0  5
   0  1  4  1  0  4  5  1  3  2  4  3  8  4  3  3  1  9  0  2  2  2  1  1
   1  1  1  2  2  2  4  2  3  2  1  3  6  5  3  1  3  3  1  2  2  1  2  2
   1  2  2 10  4  4  5  2  2  1  3  0  4  1  0  5  3  6  3  3  4  1  1  4
   7  5  0  3  1  0  6  3  2  1  2  0  2  5  0  3  3  1  4  5  7  7  1  1
   4  2  0  5  0  0  0  0]]
[2.]
[[ 4  1  6  1  5  7  2  7  3  9  0  0  1  4  2 11  0  6 10  2  3  0  3  4
   3  5  4  4  2 13  4  0  9  5  2  0  0  4  3  3  2  1  2  2  5  4  1  3
   1  4  1  1  4  4 11 10  6  8  3  5  2 10  3  2  0  2  4  8  6  0  1  4
   4 10  4  3  0  1  0  4  5  5 12  0  7  6  1  0  0  6  6  8  6  5  1  6
   1  7  5  4  6  6  2  5  7  8  2  0  8  0  6  3  4  4  3  5  4  1  5  0
   2 

[[ 0  2  4  2  0  1  0 13  3  0  0  0  2  5  6  3  0  4  3  0  6  0  3  0
   4  1  4  0  3  3  3  0  5  1  0  3  5  0  1  3  0  1  3  0  0  3  2  1
   0  1  3  5  1  3  0  0  6  3  1  0  4  6  1  9  4  1  1  0  2  6  4  3
   3  2  3  4  1  6  0  0  7  0  0  3  0  0  0  9  3  3  0  1  5  6  1  5
   3  1  3  2  0  5  0  1  4  4  1  1  6  1  4  6  2  0  0  3  0  0  1  3
   3  0  2  6  3  9  1  4  1  4  0  2  3  0  1  8  3  2  4  0  8  1  6  1
   0  0  0  1  0 11  1  3  7  1  2  1  4  0  1  2  0  5  2  1  1  5  0  5
   0  5  2  0  3  5  7  3  1  1  3  0  0  5  1  2  0  0  3  4  0  5  1  1
   3  2  2  0  0  0  9  1]]
[2.]
[[ 0  2  0  6  0  2  1  1  0  1  0  0  1  0  1  1  0  5  0  3  0  0  5  1
   4  3  3  1  0  0  1  0  3  0  0  0  0  0  0  6  1  5  0  0  0  1  0  1
   0  0  0  2  5  0  1  1  0  5  0  1  4  4  0  2  0  6  2  0  0  3  2  0
   0  0  3  1  0  0  0  0  2  3  0  2  1  2  0 10  0  2  2  0  5  2  0  0
   1  1  1  0  0  3  2  3  7  3  1 11  4  3  1  5  1  5  1  5  0  2  0  0
   2 